## 0. 환경 구성

### 1) 라이브러리 설치

In [5]:
#%pip install -q langchain langchain-openai langchain_community tiktoken chromadb onnxruntime

In [6]:
#%pip show onnxruntime

In [7]:
#%pip install -q langchain langchain-openai langchain_community faiss-cpu

### 2) OpenAI 인증키 설정
https://openai.com/

In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

sk


#### RAG 파이프 라인
* Load Data - Text Split - Indexing - Retrieval - Generation

In [3]:
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from pprint import pprint

# 1. Load Data
loader = TextLoader("data/taxinfo.txt", encoding="utf-8")
documents = loader.load()

# 2️. Text Split
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(documents)
#print(split_docs)

# 3️. Indexing (벡터 저장)
vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())
# 로컬 파일로 저장
vectorstore.save_local("faiss_index")

# 4️. Retrieval (유사 문서 검색)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
# **질문(쿼리)**에 대해 유사한 문서를 검색하는 역할
retrieved_docs = retriever.invoke("소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?")
#print(retrieved_docs)

# 5️. Generation (LLM 응답 생성)
llm = ChatOpenAI(model="gpt-4o")
context = "\n\n".join([doc.page_content for doc in retrieved_docs])
#print(context)

response_context = llm.invoke(f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요? 관련 정보: {context}")
print('context 적용한 결과')
pprint(response_context.content)

response = llm.invoke(f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?")
print('context 적용하지 않은 결과')
pprint(response.content)



context 적용한 결과
('소득세법 제12조에 따르면, 다음과 같은 소득들은 비과세 소득으로 간주되어 소득세가 부과되지 않습니다:\n'
 '\n'
 '1. **공익신탁의 이익**: 「공익신탁법」에 따른 공익신탁에서 발생하는 이익은 비과세 대상입니다.\n'
 '\n'
 '2. **사업소득** 중 비과세되는 소득:\n'
 '   - **논ㆍ밭을 작물 생산에 이용해서 발생하는 소득**: 이러한 소득은 작물 생산과 관련된 소득으로 비과세 대상에 포함됩니다.\n'
 '   - **주택임대소득**: 1개의 주택을 소유한 자가 받는 일부 주택임대소득은 비과세됩니다. 하지만 이는 주택의 기준시가가 12억원을 '
 '초과하지 않을 경우에 해당되며, 국외에 있는 주택의 임대소득은 제외됩니다. 또한, 해당 과세기간에 특정한 조건을 충족하는 경우도 '
 '해당됩니다.\n'
 '   - **농어가부업소득**: 대통령령으로 정하는 농어가부업소득도 비과세 대상입니다.\n'
 '   - **전통주의 제조 소득**: 전통주 제조에서 발생하는 소득으로 대통령령에 따른 소득도 비과세입니다.\n'
 '   - **임목의 벌채 또는 양도로 발생하는 소득**: 조림기간이 5년 이상인 임지의 임목에 대한 벌채 또는 양도로 발생하는 소득으로 '
 '연 600만원 이하인 금액은 비과세 됩니다.\n'
 '   - **작물재배업에서 발생하는 소득**: 특정 작물재배업에서 발생하는 소득으로 대통령령에 의해 정해진 소득이 비과세됩니다.\n'
 '\n'
 '각 항목의 구체적인 요건 및 적용 방법은 대통령령 등 관련 법령에 따라 구체적으로 정해집니다. 이러한 비과세 소득 항목들은 개정 및 '
 '변동될 수 있으므로, 관련 법령을 확인하는 것이 좋습니다.')
context 적용하지 않은 결과
('소득세법에서 비과세소득으로 간주되는 소득은 특정 요건을 충족하는 경우 과세 대상에서 제외됩니다. 일반적으로 비과세소득에는 다음과 같은 '
 '항목들이 포함될 수 있습니다.\n'
 '\n'
 '1. **국내외 정부 또는 지

### 개선한 Source - version1

In [2]:

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from pprint import pprint

# 1. 데이터 로드 (기존과 동일)
loader = TextLoader("data/taxinfo.txt", encoding="utf-8")
documents = loader.load()

# 2. 텍스트 분할 개선
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 크기 증가
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""],  # 자연스러운 분할을 위한 구분자
    length_function=len,
    is_separator_regex=False,
)
split_docs = splitter.split_documents(documents)

# 3. 인덱싱 (벡터 저장)
vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())
vectorstore.save_local("faiss_index")

# 4. 검색 개선
retriever = vectorstore.as_retriever(
    search_type="mmr",  # 최대 다양성 검색
    search_kwargs={"k": 5, "fetch_k": 10}  # 더 많은 결과 검색
)

# 5. 프롬프트 엔지니어링
def generate_prompt(query, context):
    return f"""다음은 소득세법 비과세소득 관련 조항입니다. 문맥을 고려하여 질문에 답변하세요.

[관련 조항]
{context}

[질문]
{query}

[답변 요구사항]
- 비과세소득 유형을 계층적으로 구분하여 설명
- 각 항목별 구체적인 조건 명시
- 법조문의 항, 호, 목 번호를 포함
- 500자 이내로 간결하게 요약"""

# 검색 및 응답 생성
query = "소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?"
retrieved_docs = retriever.invoke(query)
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

llm = ChatOpenAI(model="gpt-4o", temperature=0.3)  # 창의성 낮춤
response = llm.invoke(generate_prompt(query, context))

print('개선된 결과:')
pprint(response.content)

개선된 결과:
('소득세법 제12조에 따르면 비과세소득은 다음과 같이 구분됩니다:\n'
 '\n'
 '1. **공익신탁의 이익**: 「공익신탁법」에 따른 공익신탁의 이익 (제12조 1항).\n'
 '\n'
 '2. **사업소득**: \n'
 '   - 논ㆍ밭을 작물 생산에 이용해 발생하는 소득 (제12조 2항 가목).\n'
 '   - 1주택 소유자의 주택임대소득(기준시가 12억원 초과 주택 및 국외 주택 제외) (제12조 2항 나목).\n'
 '   - 대통령령으로 정하는 농어가부업소득, 전통주 제조 소득, 조림기간 5년 이상 임지의 임목 벌채/양도 소득(연 600만원 이하) 등 '
 '(제12조 2항 다~사목).\n'
 '\n'
 '3. **근로소득 및 퇴직소득**:\n'
 '   - 국외 및 북한지역 근로 급여, 국가/지자체/사용자 부담 보험료, 특정 연장/야간/휴일근로 급여 (제12조 3항 거~어목).\n'
 '\n'
 '4. **기타소득**:\n'
 '   - 보훈급여금, 상금, 부상, 직무발명보상금 등 (제12조 5항 가~자목).\n'
 '\n'
 '각 항목은 관련 법령 및 대통령령에 따라 구체적인 조건이 적용됩니다.')


### 개선한 Source - version2


In [4]:
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from pprint import pprint

# 1. Load Data
loader = TextLoader("data/taxinfo.txt", encoding="utf-8")
documents = loader.load()

print("=== 원본 문서 길이 ===")
print(f"전체 문서 길이: {len(documents[0].page_content)} 글자")

# 2. Text Split 개선
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,  # 500 → 800 (법령 조항이 길어서)
    chunk_overlap=150,  # 50 → 150 (맥락 보존 강화)
    separators=["\n\n", "\n", ". ", " ", ""]  # 법령 구조에 맞는 분리자
)
split_docs = splitter.split_documents(documents)

print(f"분할된 문서 수: {len(split_docs)}개")
print("=== 분할 예시 ===")
for i, doc in enumerate(split_docs[:3]):
    print(f"Chunk {i+1} ({len(doc.page_content)}글자): {doc.page_content[:100]}...")

# 3. Indexing
vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())
vectorstore.save_local("faiss_index")

# 4. Retrieval 개선
retriever = vectorstore.as_retriever(
    search_type="similarity", 
    search_kwargs={"k": 6}  # 2 → 6으로 증가
)

query = "소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?"
retrieved_docs = retriever.invoke(query)

print(f"\n=== 검색된 문서 ({len(retrieved_docs)}개) ===")
for i, doc in enumerate(retrieved_docs):
    print(f"문서 {i+1}: {doc.page_content[:200]}...")
    print("---")

# 5. Generation - 개선된 프롬프트
llm = ChatOpenAI(model="gpt-4o", temperature=0)
context = "\n\n".join([f"[문서 {i+1}]\n{doc.page_content}" for i, doc in enumerate(retrieved_docs)])

# 개선된 프롬프트 - 더 구체적인 지시사항
improved_prompt = f"""
당신은 세무 전문가입니다. 아래 소득세법 제12조 조항을 바탕으로 질문에 답변해주세요.

질문: {query}

법령 조항:
{context}

다음 형식으로 답변해주세요:
1. 비과세소득의 정의
2. 주요 비과세소득 항목들을 다음과 같이 분류:
   - 사업소득 관련
   - 근로소득/퇴직소득 관련  
   - 연금소득 관련
   - 기타소득 관련
3. 각 항목별 구체적인 조건이나 한도액 명시

답변은 법조문을 인용하면서 구체적으로 작성해주세요.
"""

# 비교용 - 기존 방식
simple_prompt = f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요? 관련 정보: {context}"

print("\n=== 개선된 프롬프트로 답변 ===")
response_improved = llm.invoke(improved_prompt)
pprint(response_improved.content)

print("\n" + "="*50)
print("=== 기존 프롬프트로 답변 ===")
response_simple = llm.invoke(simple_prompt)
pprint(response_simple.content)

# 추가 개선: 다른 검색 방식 시도
print("\n" + "="*50)
print("=== 검색 방식 개선 테스트 ===")

# MMR(Maximum Marginal Relevance) 검색 - 다양성 확보
retriever_mmr = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 6, "fetch_k": 20}
)
retrieved_docs_mmr = retriever_mmr.invoke(query)
context_mmr = "\n\n".join([f"[문서 {i+1}]\n{doc.page_content}" for i, doc in enumerate(retrieved_docs_mmr)])

response_mmr = llm.invoke(f"""
{query}

법령 조항 (MMR 검색):
{context_mmr}

위 법령을 바탕으로 비과세소득 항목들을 체계적으로 정리해주세요.
""")

print("=== MMR 검색 결과 ===")
pprint(response_mmr.content)

=== 원본 문서 길이 ===
전체 문서 길이: 4971 글자
분할된 문서 수: 8개
=== 분할 예시 ===
Chunk 1 (738글자): 제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2....
Chunk 2 (636글자): 다. 대통령령으로 정하는 농어가부업소득
    라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득
    마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양...
Chunk 3 (792글자): 라. 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받는 요양보상금, 휴업보상금, 상병보상금(傷病補償金), 일시보상금, 장해보상금, 유족보상금, 행방불명보상금, ...

=== 검색된 문서 (6개) ===
문서 1: 제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 2...
---
문서 2: 2) 대학의 교직원 또는 대학과 고용관계가 있는 학생이 소속 대학에 설치된 「산업교육진흥 및 산학연협력촉진에 관한 법률」 제25조에 따른 산학협력단(이하 이 조에서 “산학협력단”이라 한다)으로부터 같은 법 제32조제1항제4호에 따라 받는 보상금
    저. 대통령령으로 정하는 복리후생적 성질의 급여
4. 연금소득 중 다음 각 목의 어느 하나에 해당하는 소득...
---
문서 3: 나. 「국가보안법」에 따라 받는 상금과 보로금
    다. 「상훈법」에 따른 훈장과 관련하여 받는 부상(副賞)이나 그 밖에 대통령령으로 정하는 상금과 부상
    라. 종업원등 또는 대학의 교직원이 퇴직한 후에 사용